# IMPORTS

In [1]:
import os
import torch
import torchaudio

In [2]:
import sounddevice as sd

In [5]:
from importlib.machinery import SourceFileLoader
mi_clasificador = SourceFileLoader('clasificador_padre', '../libs/clasificador_padre.py').load_module()
model_path = "./save_old_ow/state/ss_1.pt"


CARGO EL MODELO

In [6]:
clasificador = mi_clasificador.onlyWoman_MFCC_05_N2(['Ariana Grande', 'Katy Perry', 'Taylor Swift'])
clasificador.load_state_dict(torch.load(model_path))
clasificador.eval()

onlyWoman_MFCC_05_N2(
  (inPut): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
  )
  (chanelUp): Sequential(
    (0): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): ReLU()
    (2): Conv2d(4, 16, kernel_size=(1, 7), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(16, 32, kernel_size=(1, 7), stride=(1, 1))
    (5): ReLU()
    (6): Conv2d(32, 64, kernel_size=(1, 7), stride=(1, 2))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (justTime): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 7), stride=(1, 2))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 7), stride=(1, 2))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, a

#PARAMETROS PARA PROCESAR EL AUDIO

In [7]:
#parametros para los calculos del mell
TARGET_SR = 16000  # Normalmente el audio se sule usar a 16k aunque encontre papers que trabajan a 22050 o a 22k (ver V2)
N_FFT = 512  #muestras de la fft
W_LEN = 400  # Numero de muestras para la ventan de la  fft (seg_de_ventan *sr) 
H_LEN = 160 # paso de la ventana entre una fft y la siguiente (paso * sr)
N_MELS = 26
N_MFCC = 13

CHANNELS = 1
DUR_SAMPLE = 10 # 10 segundos


In [8]:
MFCCCalculator = torchaudio.transforms.MFCC(sample_rate = TARGET_SR,
                                            n_mfcc = int(N_MFCC),
                                            dct_type = 2,
                                            norm = 'ortho',
                                            log_mels = False,
                                            melkwargs = {"n_fft": N_FFT, "hop_length": H_LEN, "n_mels": N_MELS, "center": False},)

In [9]:
dic = {0:"Ari", 1:"Katy", 2:"Taylor"}

In [15]:
record = sd.rec(int(DUR_SAMPLE*TARGET_SR), samplerate=TARGET_SR, channels=CHANNELS, dtype='float32', blocking= True)
record_tensor = torch.from_numpy(record.T)
mfcc = MFCCCalculator(record_tensor)
res = clasificador(mfcc)
pred = dic[int(torch.argmax(res))]
print(pred)


Ari


# TOMA DE MUESTRA Y PROCESADO DEL AUDIO

In [17]:
record_tensor.shape

torch.Size([1, 160000])

In [16]:
mfcc.shape

torch.Size([1, 13, 997])